In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import os
import cv2
import numpy as np
import urllib.request

df = pd.read_csv("fusion_colonne.csv")
df.head()

,Unnamed: 0,ID,Image,Nom_Structure,Angle_Vue,Date,taskName,featureId,schemaId,color,title,value,top,left,height,width,instanceURI
0,0,clgdouz2d117y070e33998k8d,SH001H00200219.jpg,SH001,H00,20/02/19,Done,clgdqaods0001356jz5hfhr3i,clgdq9gr40bqh072l1c1v6e1n,#1CE6FF,Acropora,acropora,694.0,604.0,53.0,84.0,https://api.labelbox.com/masks/feature/clgdqao...
1,1,clgdouz2d117y070e33998k8d,SH001H00200219.jpg,SH001,H00,20/02/19,Done,clgdr5bj10001356jrotvujk1,clgdq9gr40bqh072l1c1v6e1n,#1CE6FF,Acropora,acropora,503.0,462.0,49.0,65.0,https://api.labelbox.com/masks/feature/clgdr5b...
2,2,clgdouz2d117y070e33998k8d,SH001H00200219.jpg,SH001,H00,20/02/19,Done,clgdr5fjr0004356jjpbi92y5,clgdq9gr40bqh072l1c1v6e1n,#1CE6FF,Acropora,acropora,409.0,368.0,57.0,55.0,https://api.labelbox.com/masks/feature/clgdr5f...
3,3,clgdouz2d117y070e33998k8d,SH001H00200219.jpg,SH001,H00,20/02/19,Done,clgdr5nzc0007356jwb7ipee3,clgdq9gr40bqh072l1c1v6e1n,#1CE6FF,Acropora,acropora,496.0,283.0,51.0,55.0,https://api.labelbox.com/masks/feature/clgdr5n...
4,4,clgdouz2d117y070e33998k8d,SH001H00200219.jpg,SH001,H00,20/02/19,Done,clgdr5r1d000a356j5p10c1zy,clgdq9gr40bqh072l1c1v6e1n,#1CE6FF,Acropora,acropora,568.0,366.0,55.0,55.0,https://api.labelbox.com/masks/feature/clgdr5r...


In [2]:
data_dir =  r"C:/Users/utilisateur/Documents/maldives-corals/Frame-segmentation/"
pictures_path = data_dir + r"unet_p6/"
masks_path = data_dir + r'/annotations/trimaps_p6/'

In [3]:
bar = df[df.value == "frame_bar"]

In [5]:
# Boucle pour chaque fichier dans la colonne 'Image' de la dataframe 'bar'
for filename in bar['Image']:
    # Vérifier si le fichier de masque correspondant n'existe pas déjà
    if not os.path.exists(masks_path + filename[:-4] + '.png'):
        # Récupérer l'URL de l'image à partir de la colonne 'instanceURI' correspondante
        url = bar.loc[bar['Image'] == filename, 'instanceURI'].values[0]
        # Ouvrir l'URL et convertir la réponse en un tableau d'octets (bytearray)
        with urllib.request.urlopen(url) as url_response:
            image_array = np.asarray(bytearray(url_response.read()), dtype="uint8")
        # Décoder le tableau d'octets en une image en niveaux de gris
        image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)
        # Appliquer un seuillage pour convertir l'image en noir et blanc
        _, bw_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
        # Inverser les couleurs de l'image
        inverted_image = cv2.bitwise_not(bw_image)
        # Enregistrer l'image normalisée en tant que fichier de masque PNG
        cv2.imwrite(masks_path + filename[:-4] + '.png', inverted_image)


### Selection des images correspondantes aux Masks

Pour que cela fonction il faut que vos dossiers ressemble à ça :
- Frame-segmentation
    - annotations
        - trimaps_p6
            - les photos des masks.png
    - unet_p6
        - les photos de coraux qui correspondent aux masks.jpg
    - CORAL_FRAMES
        - SH001
            - photos des strctures SH001.jpg
        - SH002
            - photos des strctures SH002.jpg
        - ...
            - ...
        - SH048
            - photos des structures SH048.jpg

/!\ Attention le code ci-dessous va supprimer les photos qui n'ont pas de correspondance avec les masks. Donc les structures qui n'ont pas de masks seront supprimer, les photos restantes devront être ranger manuellement dans le dossier `unet_p6`.

In [ ]:
import os

# Chemin des dossiers contenant les images et les annotations
images_dir = os.path.join(os.getcwd(), "CORAL_FRAMES")
annotations_dir = os.path.join(os.getcwd(), "annotations", "trimaps_p6")

# Obtenir la liste des noms de fichiers d'annotations sans l'extension ".png"
annotation_files = [os.path.splitext(f)[0] for f in os.listdir(annotations_dir) if f.endswith(".png")]

# Créer le dossier "unet_p6" s'il n'existe pas déjà
unet_dir = os.path.join(images_dir, "unet_p6")
if not os.path.exists(unet_dir):
    os.mkdir(unet_dir)
# Parcourir tous les dossiers "SHxxx" contenant les images
for subdir in os.listdir(images_dir):
    subdir_path = os.path.join(images_dir, subdir)    
    if os.path.isdir(subdir_path) and subdir.startswith("SH"):
        # Parcourir tous les fichiers image dans le dossier
        for image_file in os.listdir(subdir_path):
            # Vérifier si le fichier est une image
            if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
                image_name = os.path.splitext(image_file)[0]
                # Vérifier si le nom du fichier image correspond à un nom de fichier d'annotation
                if ' ' + image_name in annotation_files:
                    # conserver l'image
                    print("==========================================")
                    print(os.path.join(subdir_path, image_file))
                else:
                    # supprimer l'image
                    os.remove(os.path.join(subdir_path, image_file))

c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00200219.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00201114.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00210421.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00211024.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00220214.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00220729.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H00230116.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H03200219.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentation\CORAL_FRAMES\SH001\SH001H03201114.jpg
c:\Users\utilisateur\Documents\maldives-corals\Frame-segmentatio

In [6]:
from functions import prepare_pictures

# Resize pictures and masks, adds padding, and stores them in out_path
out_path = data_dir + r"unet_padded/"

prepare_pictures(masks_path, pictures_path, out_path, target_width = 200)

C:/Users/utilisateur/Documents/maldives-corals/Frame-segmentation//annotations/trimaps_p6/
324 images traitées
324 masques traités


In [19]:
import numpy as np
from PIL import Image
from numpy.random import random

class CustomDataProvider:
    
    def __init__(self, data_folder, data_suffix, mask_suffix):
        self.data_folder = data_folder
        self.data_suffix = data_suffix
        self.mask_suffix = mask_suffix
    
    def load_data(self, id):
        data_file = self.data_folder + '/' + id + self.data_suffix
        mask_file = self.data_folder + '/' + id + self.mask_suffix
        
        data = np.array(Image.open(data_file))
        mask = np.array(Image.open(mask_file))
        
        return data, mask
    
    def _post_process(self, data, labels):
        # Flip
        if random() > 0.5:
            data = np.fliplr(data)
            labels = np.fliplr(labels)
        
        # Fix ratio
        (h, w) = data.shape[:2]
        new_h = int(0.75 * w)
        data = np.pad(data, ((0, new_h - h), (0, 0), (0, 0)), mode='reflect')
        labels = np.pad(labels, ((0, new_h - h), (0, 0), (0, 0)), mode='reflect')
        
        # Rotate
        angle = int(random() * 360)
        data = Image.fromarray(data)
        labels = Image.fromarray(labels)
        data = data.rotate(angle, resample=Image.BICUBIC, fillcolor='reflect')
        labels = labels.rotate(angle, resample=Image.BICUBIC, fillcolor='reflect')
        data = np.array(data)
        labels = np.array(labels)
        
        # Crop to original size plus add padding to account for cropping
        (new_h, new_w) = data.shape[:2]
        w_margins = (new_w - w) // 2, (new_w - w) // 2 + (new_w - w) % 2
        h_margins = (new_h - h) // 2, (new_h - h) // 2 + (new_h - h) % 2
        data = data[h_margins[0]:(new_h - h_margins[1]), w_margins[0]:(new_w - w_margins[1])]
        labels = labels[h_margins[0]:(new_h - h_margins[1]), w_margins[0]:(new_w - w_margins[1])]
        data = np.pad(data, ((200, 200), (200, 200), (0, 0)), mode='reflect')
        labels = np.pad(labels, ((200, 200), (200, 200), (0, 0)), mode='reflect')
        
        # Return processed data and labels
        return data, labels

In [21]:
from tf_unet import unet, util, image_util
# from functions import CustomDataProvider

#preparing data loading
data_provider = CustomDataProvider(data_folder=r"C:/Users/utilisateur/Documents/maldives-corals/Frame-segmentation/unet_padded",
                                    data_suffix=".png",
                                    mask_suffix='_mask.png')

                                             

#setup & training
net = unet.Unet(layers=6, features_root=64, channels=3, n_class=2)
trainer = unet.Trainer(net, batch_size = 10, optimizer='adam')

path = trainer.train(data_provider,
                     "/home/mdc/Desktop/Frame segmentation/unet_output", 
                     training_iters=10,
                     epochs=1000,
                     restore=True,
                     next_step=28672)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, Activation, UpSampling3D, Concatenate
from tensorflow.keras.models import Model

# Création de l'architecture du modèle
def get_model(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    x1 = Conv3D(16, 3, padding="same")(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Activation("relu")(x1)
    
    x2 = Conv3D(32, 3, strides=2, padding="same")(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation("relu")(x2)
    
    x3 = Conv3D(64, 3, strides=2, padding="same")(x2)
    x3 = BatchNormalization()(x3)
    x3 = Activation("relu")(x3)
    
    x4 = Conv3D(128, 3, strides=2, padding="same")(x3)
    x4 = BatchNormalization()(x4)
    x4 = Activation("relu")(x4)
    
    x5 = Conv3D(256, 3, strides=2, padding="same")(x4)
    x5 = BatchNormalization()(x5)
    x5 = Activation("relu")(x5)
    
    # Decoder
    x = UpSampling3D()(x5)
    x = Concatenate()([x, x4])
    x = Conv3D(128, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D()(x)
    x = Concatenate()([x, x3])
    x = Conv3D(64, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D()(x)
    x = Concatenate()([x, x2])
    x = Conv3D(32, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D()(x)
    x = Concatenate()([x, x1])
    x = Conv3D(16, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    outputs = Conv3D(1, 1, activation="sigmoid")(x)
    
    # Création du modèle
    model = Model(inputs, outputs)
    
    return model

# Définition de la fonction de perte
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Compilation du modèle
model = get_model(input_shape=(128, 128, 128, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss=dice_loss, metrics=[dice_coefficient])


c:\Users\utilisateur\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [29]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms


# Define dataset and data loader
data_folder = "C:/Users/utilisateur/Documents/maldives-corals/Frame-segmentation/unet_padded"
data_transforms = transforms.Compose([
    transforms.ToTensor(),
])
dataset = CustomDataset(data_folder, data_transforms)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Define VNet model and optimizer
model = VNet(num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define loss function
loss_fn = nn.CrossEntropyLoss()

# Train the model
for epoch in range(1000):
    epoch_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {epoch_loss / len(dataloader)}")

# Save the model
torch.save(model.state_dict(), "vnet_model.pth")


ModuleNotFoundError: No module named 'vnet'

In [27]:
import torch
print(torch.__version__)

1.13.1+cpu
